In [1]:
# use sqlalchemy to write a orm query to a sqlite database

In [20]:
from sqlalchemy import create_engine, text
import pandas as pd

In [21]:
# Create a SQLite engine
engine = create_engine('sqlite:///../../data/AllPrintings.sqlite')

# Test the connection
try:
    with engine.connect() as connection:
        result = connection.execute(text("SELECT 1"))
        print("Connection successful:", result.scalar() == 1)
except Exception as e:
    print("Connection failed:", str(e))


Connection successful: True


Here we'll use the 'text' approach

In [25]:
# Define the SQL command to create the view
drop_view = """
DROP VIEW IF EXISTS standard_cards_view;
"""

create_view_sql = """
CREATE VIEW standard_cards_view AS
SELECT
    cl.standard, 
    c.name, 
    c.setCode, 
    cs.releaseDate,
    c.number, 
    c.layout, 
    c.*
FROM cards AS c
JOIN cardLegalities AS cl ON cl.uuid = c.uuid
JOIN cardPurchaseUrls AS cp ON cp.uuid = c.uuid
JOIN sets AS cs ON cs.code = c.setCode
WHERE cl.standard = 'Legal' 
    AND c.setCode = 'OTJ'
    AND c.isPromo IS NULL 
    AND c.borderColor = 'black' 
    AND c.isReprint IS NULL 
    AND c.promoTypes IS NULL 
ORDER BY c.name ASC, cs.releaseDate ASC, c.power DESC;
"""

# Execute the SQL command
with engine.connect() as connection:
    connection.execute(text(drop_view))
    connection.execute(text(create_view_sql))

print("View 'standard_cards_view' created successfully.")

View 'standard_cards_view' created successfully.


In [26]:

# Define the SQL query
sql_query = """
WITH ranked_cards AS (
    SELECT
        *,
        ROW_NUMBER() OVER (PARTITION BY name ORDER BY releaseDate ASC, power DESC) AS rn
    FROM standard_cards_view
)
SELECT
    name, 
    setCode, 
    releaseDate,
    number, 
    layout,
    availability,
    power, toughness,
    colorIdentity, colors,
    types, subtypes, supertypes,
    manaCost, manaValue,
    edhrecRank, edhrecSaltiness,
    text, flavorText
FROM ranked_cards
WHERE rn = 1
ORDER BY name ASC;
"""

# Execute the query and load the results into a DataFrame
with engine.connect() as connection:
    df = pd.read_sql_query(sql_query, connection)

# Display the DataFrame
df

,name,setCode,releaseDate,number,layout,availability,power,toughness,colorIdentity,colors,types,subtypes,supertypes,manaCost,manaValue,edhrecRank,edhrecSaltiness,text,flavorText
0,Abraded Bluffs,OTJ,2024-04-19,251,normal,"arena, mtgo, paper",None,None,"R, W",,Land,Desert,,None,0.0,5898,None,Abraded Bluffs enters the battlefield tapped.\...,"""The wind passing through the grooves makes a ..."
1,Akul the Unrepentant,OTJ,2024-04-19,189,normal,"arena, mtgo, paper",5,5,"B, R","B, R",Creature,"Scorpion, Dragon, Rogue",Legendary,{B}{B}{R}{R},4.0,12398,None,"Flying, trample\nSacrifice three other creatur...","Crackling with rage and stolen thunder, he dri..."
2,Aloe Alchemist,OTJ,2024-04-19,152,normal,"arena, mtgo, paper",3,2,G,G,Creature,"Plant, Warlock",,{1}{G},2.0,17373,None,"Trample\nWhen Aloe Alchemist becomes plotted, ...",None
3,Ambush Gigapede,OTJ,2024-04-19,77,normal,"arena, mtgo, paper",6,2,B,B,Creature,Insect,,{4}{B}{B},6.0,21172,None,Flash\nWhen Ambush Gigapede enters the battlef...,"""First rule of crossing Blacksnag Bog: if you ..."
4,Ankle Biter,OTJ,2024-04-19,153,normal,"arena, mtgo, paper",1,1,G,G,Creature,Snake,,{G},1.0,12461,None,Deathtouch,"""Little tip, newcomer. Give your boots a shake..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
254,Vraska Joins Up,OTJ,2024-04-19,236,normal,"arena, mtgo, paper",None,None,"B, G","B, G",Enchantment,,Legendary,{B}{G},2.0,6324,None,"When Vraska Joins Up enters the battlefield, p...",Letting Oko believe he held all the cards prov...
255,"Vraska, the Silencer",OTJ,2024-04-19,237,normal,"arena, mtgo, paper",3,3,"B, G","B, G",Creature,"Gorgon, Assassin",Legendary,{1}{B}{G},3.0,8921,None,Deathtouch\nWhenever a nontoken creature an op...,None
256,Wanted Griffin,OTJ,2024-04-19,38,normal,"arena, mtgo, paper",3,2,W,W,Creature,Griffin,,{3}{W},4.0,23525,None,"Flying\nWhen Wanted Griffin dies, create a 1/1...",Many a would-be captor has gotten close enough...
257,Wrangler of the Damned,OTJ,2024-04-19,238,normal,"arena, mtgo, paper",1,4,"U, W","U, W",Creature,"Human, Soldier",,{3}{W}{U},5.0,22197,None,"Flash\nAt the beginning of your end step, if y...","""Unfinished business goes both ways, pal."""


In [27]:
df.to_feather('../../data/OTJ/card.feather')